In [5]:
import torch.utils.data as data
import cv2
import sys
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import tqdm as notebook_tqdm

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable
from Utils import *
from Dataset import *

class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN,self).__init__()
        resnet = models.resnet34(pretrained = True)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum = 0.01)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0.0, 0.02)
        self.linear.bias.data.fill_(0)

    def forward(self, images):
        features = self.resnet(images)
        features = Variable(features.data)
        features = features.view(features.size(0), -1)
        if images.shape[0] < 2:
            features = self.linear(features)
            return features
        features = self.bn(self.linear(features))
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(DecoderRNN, self).__init__()
        self.n_layers = num_layers
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.GRU(embed_size*2, hidden_size, num_layers, batch_first = True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.init_weights()

    def init_weights(self):
        self.embed.weight.data.uniform_(-0.1, 0.1)
        self.linear.weight.data.uniform_(-0.1, 0.1)
        self.linear.bias.data.fill_(0)

    def forward(self, features, captions, hidden):
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1).repeat(1, embeddings.shape[1], 1), embeddings), 2)
        output, hidden = self.lstm(embeddings, hidden)
        outputs = self.linear(output)
        return outputs, hidden
    
    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset_init = Dataset('data/')

In [9]:
encoder = torch.load(r'C:\Users\hp\Desktop\WireFrameCodeGen\EncoderWeightsTensor(0.0616).pt')
decoder = torch.load(r'C:\Users\hp\Desktop\WireFrameCodeGen\DecoderWeightsTensor(0.0616).pt')

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has ch

In [11]:
from nltk.translate.bleu_score import sentence_bleu

decoded_words = []
start_text = ' '
hidden = decoder.init_hidden()
image = load_val_images('val/')[0]
image = Variable(torch.FloatTensor([image]))
predicted = ' '

C:\Users\hp\AppData\Local\Temp\ipykernel_13236\767215093.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  image = Variable(torch.FloatTensor([image]))


In [12]:
for di in range(9999):
    sequence = dataset_init.tokenizer.texts_to_sequences([start_text])[0]
    decoder_input = Variable(torch.LongTensor(sequence)).view(1,-1)
    features = encoder(image)

    outputs,hidden = decoder(features,decoder_input,hidden)
    topv, topi = outputs.data.topk(1)
    ni = topi[0][0][0]
    word = word2idx(ni,dataset_init.tokenizer)
    if word is None:
        continue
    predicted += word + ' '
    start_text = word
    print(predicted)
    if word == '':
        break

AttributeError: 'BatchNorm2d' object has no attribute 'track_running_stats'

In [ ]:
original_gui = load_doc('/content/drive/MyDrive/val/2BC033FD-F097-463B-98A8-C1C9CE50B478.gui')
original_gui = ' '.join(original_gui.split())
original_gui = original_gui.replace(',', ' ,')
original_gui = original_gui.split()

btns_to_replace = ['btn-green','btn-red']
normalized_original_gui = ['btn-orange' if token in btns_to_replace else token for token in original_gui]
normalized_original_gui = ['btn-active' if token == 'btn-inactive' else token for token in normalized_original_gui]

generated_gui = predicted.split()

normalized_generated_gui = ['btn-orange' if token in btns_to_replace else token for token in generated_gui]
normalized_generated_gui = ['btn-active' if token == 'btn-inactive' else token for token in normalized_generated_gui]

print(sentence_bleu([normalized_original_gui],normalized_generated_gui))